In [4]:
import numpy as np

def add_noise(keypoints_seq, noise_level=0.01):
    noise = np.random.normal(0, noise_level, keypoints_seq.shape)
    return keypoints_seq + noise
def scale_keypoints(keypoints_seq, scale_range=(0.9, 1.1)):
    scale = np.random.uniform(*scale_range)
    return keypoints_seq * scale
def translate_keypoints(keypoints_seq, translate_range=(-0.05, 0.05)):
    translation = np.random.uniform(*translate_range, size=(1, keypoints_seq.shape[1]))
    return keypoints_seq + translation
def rotate_keypoints(sequence, angle_range=(-15, 15)):
    angle = np.radians(np.random.uniform(*angle_range))
    cos, sin = np.cos(angle), np.sin(angle)
    rotation_matrix = np.array([[cos, -sin], [sin, cos]])

    sequence = sequence.copy()
    for i in range(sequence.shape[0]):
        for j in range(0, sequence.shape[1] - 1, 3):  # x, y, z
            xy = sequence[i, j:j+2]
            rotated_xy = np.dot(rotation_matrix, xy)
            sequence[i, j:j+2] = rotated_xy
    return sequence
def augment_keypoints(keypoints_seq):
    # Apply multiple augmentations sequentially
    augmented = add_noise(keypoints_seq, noise_level=0.02)
    augmented = scale_keypoints(augmented)
    augmented = translate_keypoints(augmented)
    augmented = rotate_keypoints(augmented)
    return augmented

In [ ]:
AUGMENTATIONS_PER_VIDEO = 5 # How many augmented samples per original
OUTPUT_DIR = 'data/numpy'
DUPLI_DIR='data/Static'
for gesture_folder in os.listdir(DUPLI_DIR):
    gesture_path = os.path.join(OUTPUT_DIR, gesture_folder)
    if not os.path.isdir(gesture_path):
        continue
    
    npy_files = glob(os.path.join(gesture_path, '*.npy'))
    for npy_file in npy_files:
        if '_aug' in npy_file:
            continue  # Skip already augmented files

        original_seq = np.load(npy_file)
        
        for i in range(AUGMENTATIONS_PER_VIDEO):
            augmented_seq = augment_keypoints(original_seq)
            # augmented_seq = np.clip(augmented_seq, 0, 1)  # If your keypoints are normalized
            base_name = os.path.splitext(os.path.basename(npy_file))[0]
            save_aug_path = os.path.join(gesture_path, f"{base_name}_aug{i+1}.npy")
            np.save(save_aug_path, augmented_seq)

In [ ]:
# %%
import numpy as np
import os
from glob import glob
from sklearn.model_selection import train_test_split

KEYPOINTS_DIR = 'data/numpy'
GESTURES = sorted(os.listdir(KEYPOINTS_DIR))  # e.g., ['hello', 'thank_you']
label_map = {gesture: idx for idx, gesture in enumerate(GESTURES)}

X, y = [], []

for gesture in GESTURES:
    print(gesture)
    gesture_path = os.path.join(KEYPOINTS_DIR, gesture)
    npy_files = glob(os.path.join(gesture_path, '*.npy'))
    
    for npy_file in npy_files:
        keypoints = np.load(npy_file)
        if keypoints.shape == (25, 126):
            # Add normalized time index to each frame
            time_indices = np.linspace(0, 1, 25).reshape(25, 1)
            keypoints_with_time = np.concatenate([keypoints, time_indices], axis=1)  # (30, 127)
            X.append(keypoints_with_time)
            y.append(label_map[gesture])


X = np.array(X)
y = np.array(y)

print(f'Dataset size: {X.shape}, Labels: {y.shape}')
print(f'Classes: {label_map}')

0
1
2
3
4
5
6
7
8
9
A
Alright
Animal
B
Beautiful
Bed
Bedroom
Bird
Black
Blind
C
Cat
Chair
Colour
Cow
D
Daughter
Deaf
Dog
Door
Dream
E
F
Father
Fish
Friday
G
Good Morning
Good night
Grey
H
Happy
He
Hello
Horse
How are you
I
Ii
It
J
K
L
Loud
M
Monday
Mother
Mouse
N
O
Orange
P
Parent
Pink
Pleased
Q
Quiet
R
S
Sad
Saturday
She
Son
Sunday
T
Table
Thank you
Thursday
Today
Tuesday
U
Ugly
V
W
Wednesday
White
Window
X
Y
You
Z
Dataset size: (44268, 25, 127), Labels: (44268,)
Classes: {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'A': 10, 'Alright': 11, 'Animal': 12, 'B': 13, 'Beautiful': 14, 'Bed': 15, 'Bedroom': 16, 'Bird': 17, 'Black': 18, 'Blind': 19, 'C': 20, 'Cat': 21, 'Chair': 22, 'Colour': 23, 'Cow': 24, 'D': 25, 'Daughter': 26, 'Deaf': 27, 'Dog': 28, 'Door': 29, 'Dream': 30, 'E': 31, 'F': 32, 'Father': 33, 'Fish': 34, 'Friday': 35, 'G': 36, 'Good Morning': 37, 'Good night': 38, 'Grey': 39, 'H': 40, 'Happy': 41, 'He': 42, 'Hello': 43, 'Horse': 44, 'How ar

In [2]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, Masking
from tensorflow.keras.utils import to_categorical

In [4]:
num_classes = len(np.unique(y_train))
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_test_cat = to_categorical(y_test, num_classes=num_classes)

In [5]:
from collections import Counter
labels = [np.argmax(y) for y in y_train_cat]
print(Counter(labels))  # Ensure balanced class counts


Counter({46: 773, 58: 432, 47: 432, 49: 432, 6: 408, 25: 408, 20: 408, 2: 408, 81: 408, 3: 408, 10: 408, 89: 408, 66: 408, 5: 408, 40: 408, 87: 408, 79: 408, 51: 408, 9: 408, 32: 408, 73: 408, 67: 408, 13: 408, 1: 408, 7: 408, 82: 408, 57: 408, 31: 408, 36: 408, 53: 408, 0: 408, 60: 408, 86: 408, 64: 408, 4: 408, 50: 408, 8: 408, 63: 389, 37: 389, 48: 389, 45: 389, 75: 389, 88: 389, 41: 389, 11: 389, 43: 389, 19: 389, 68: 389, 70: 389, 52: 389, 14: 389, 38: 389, 65: 389, 27: 389, 42: 389, 17: 389, 18: 384, 26: 384, 71: 384, 62: 384, 59: 384, 33: 384, 84: 384, 39: 384, 61: 384, 23: 384, 55: 384, 44: 379, 24: 379, 34: 379, 56: 379, 21: 379, 12: 379, 28: 379, 35: 370, 69: 370, 77: 369, 78: 369, 54: 369, 72: 369, 83: 369, 76: 365, 74: 336, 22: 336, 29: 336, 30: 336, 16: 331, 85: 331, 15: 331, 80: 326})


In [6]:
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Masking, Bidirectional, BatchNormalization
from tensorflow.keras.models import Sequential

model = Sequential([
    Masking(mask_value=0.0, input_shape=(None, 127)),  # ← None for variable length

    Bidirectional(LSTM(128, return_sequences=True)),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(32)),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(128, activation='relu'),
    Dropout(0.2),

    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, None, 127)         0         
                                                                 
 bidirectional (Bidirection  (None, None, 256)         262144    
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, None, 128)         164352    
 onal)                                                           
                                                                 
 bidirectional_2 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                        

In [7]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=15,               # Increased from 10 to 15
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,                # More gradual decay
    patience=5,
    verbose=1,
    min_lr=1e-6                # Optional lower bound
)


history = model.fit(
    X_train, y_train_cat,
    validation_data=(X_test, y_test_cat),
    epochs=100,
    batch_size=64,
    shuffle=True,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)



Epoch 1/100
554/554 [==============================] - 517s 684ms/step - loss: 2.6027 - accuracy: 0.3047 - val_loss: 1.5193 - val_accuracy: 0.5539 - lr: 0.0010
Epoch 2/100
554/554 [==============================] - 173s 313ms/step - loss: 1.1712 - accuracy: 0.6453 - val_loss: 0.8012 - val_accuracy: 0.7530 - lr: 0.0010
Epoch 3/100
554/554 [==============================] - 76s 138ms/step - loss: 0.7819 - accuracy: 0.7594 - val_loss: 0.6438 - val_accuracy: 0.7957 - lr: 0.0010
Epoch 4/100
554/554 [==============================] - 76s 138ms/step - loss: 0.6226 - accuracy: 0.8052 - val_loss: 0.4929 - val_accuracy: 0.8362 - lr: 0.0010
Epoch 5/100
554/554 [==============================] - 76s 137ms/step - loss: 0.5117 - accuracy: 0.8393 - val_loss: 0.4039 - val_accuracy: 0.8688 - lr: 0.0010
Epoch 6/100
554/554 [==============================] - 77s 138ms/step - loss: 0.4406 - accuracy: 0.8605 - val_loss: 0.3698 - val_accuracy: 0.8815 - lr: 0.0010
Epoch 7/100
554/554 [=======================

In [8]:
model.save('fullset.h5')


C:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
